In [42]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Carregar os dados
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Para evitar o FutureWarning, se necessário
pd.set_option('future.no_silent_downcasting', True)

# Visualizar as primeiras linhas
# print(train.head())
# print(test.head())

# Verificar informações sobre os dados
print(train.info())
print(test.info())

# Imputar ou remover valores ausentes
train['Age'] = train['Age'].fillna(train['Age'].mean())
train['HomePlanet'] = train['HomePlanet'].fillna(train['HomePlanet'].mode()[0])
train['CryoSleep'] = train['CryoSleep'].fillna(train['CryoSleep'].mode()[0])
train['Cabin'] = train['Cabin'].fillna(train['Cabin'].mode()[0])
train['Destination'] = train['Destination'].fillna(train['Destination'].mode()[0])
train['VIP'] = train['VIP'].fillna(False)

test['Age'] = test['Age'].fillna(test['Age'].mean())
test['HomePlanet'] = test['HomePlanet'].fillna(test['HomePlanet'].mode()[0])
test['CryoSleep'] = test['CryoSleep'].fillna(test['CryoSleep'].mode()[0])
test['Cabin'] = test['Cabin'].fillna(test['Cabin'].mode()[0])
test['Destination'] = test['Destination'].fillna(test['Destination'].mode()[0])
test['VIP'] = test['VIP'].fillna(False)

# Remover a coluna 'Transported' de 'train' para preparar para treino
train['Transported'] = train['Transported'].map({'True': 1, 'False': 0, True: 1, False: 0})

# Concatenar train e test para garantir que as colunas sejam iguais após get_dummies
train['is_train'] = 1
test['is_train'] = 0

# Preservar 'PassengerId' para uso na submissão
test_passenger_id = test['PassengerId']

data = pd.concat([train, test], axis=0, sort=False)

# Aplicar get_dummies para variáveis categóricas
data = pd.get_dummies(data, columns=['HomePlanet', 'CryoSleep', 'Destination', 'Cabin'], drop_first=True)

# Separar de volta em train e test após transformação
train = data[data['is_train'] == 1].drop(columns=['is_train'])
test = data[data['is_train'] == 0].drop(columns=['is_train'])

# Remover colunas que não são necessárias para o modelo
train.drop(columns=['PassengerId', 'Name'], inplace=True)
test.drop(columns=['PassengerId', 'Name'], inplace=True)

# Garantir que o conjunto de teste não tenha a coluna 'Transported'
test = test.drop(columns=['Transported'], errors='ignore')

# Separar as variáveis independentes (X) e dependentes (y) para o treino
X = train.drop(columns=['Transported'])
y = train['Transported']

# Dividir o conjunto de treino em treino e validação
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo
model = RandomForestClassifier(random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de validação
y_pred = model.predict(X_valid)

# Avaliar o modelo
accuracy = accuracy_score(y_valid, y_pred)
print(f'Acurácia: {accuracy:.4f}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Co

In [43]:
# Previsões no conjunto de teste
test_predictions = model.predict(test)

# Criar o arquivo de submissão
submission = pd.DataFrame({'PassengerId': test_passenger_id, 'Transported': test_predictions})

# Salvar o arquivo de submissão
submission.to_csv('submission.csv', index=False)
print('Download Sucess')

Download Sucess
